In [55]:
DATA_PATH = 'C:\\Users\\dmytro.rybalko\\Documents\\impementations\Keras_PairCNN\\jacana-qa-naacl2013-data-results\\'
OUTPUT_PATH = 'C:\\Users\\dmytro.rybalko\\Documents\\impementations\Keras_PairCNN\\jacana-qa-naacl2013-data-results\\preprocessed_data\\'
VOCAB_PATH = OUTPUT_PATH + 'vocab.json'
EMBEDDING_PATH = OUTPUT_PATH + 'aquaint+wiki.txt.gz.ndim=50.bin'

BATCH_SIZE = 50
EPOCHS = 7
RANDOM_STATE =  42

In [2]:
from gensim.summarization.bm25 import BM25

def calculate_bm25_scores(question, candidate_set):
    model = BM25(candidate_set)
    average_idf = sum(float(val) for val in model.idf.values()) / len(model.idf)
    scores = []
    for idx, val in enumerate(candidate_set):
        scores += [model.get_score(question, idx, average_idf)]
    return scores

C:\Users\dmytro.rybalko\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
def softmax(x, axis=None):
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    return y / y.sum(axis=axis, keepdims=True)

In [4]:
def convert_bm25_score_to_relevance(scores, top_k = 3):
    scores = np.array(scores)
    probs = softmax(scores)
    if scores.shape[0] <= top_k:
        threshold_prob = probs[np.argsort(probs)][0]
    else:
        threshold_prob = probs[np.argsort(probs)[::-1][top_k - 1]]
    rel = np.array(probs >= threshold_prob, dtype = np.int16)
    return probs, rel

In [5]:
def generate_bm25_labels(qids, questions, answers):
    qids = np.array(qids)
    questions = np.array(questions)
    answers = np.array(answers)
    unique_qids = np.unique(qids)
    
    probs_full = np.empty(0)
    rel_full = np.empty(0)
    for qid in unique_qids:
        index_mask = qids == qid
        question = questions[index_mask][0]
        candidate_set = answers[index_mask]
        
        scores = calculate_bm25_scores(question, candidate_set)
        probs, rel = convert_bm25_score_to_relevance(scores)

        probs_full = np.concatenate((probs_full, probs), axis = None)
        rel_full = np.concatenate((rel_full, rel), axis = None)
    return probs_full, rel_full

In [6]:
qids, questions, answers, labels = load_data(OUTPUT_PATH + "tmp\\trec-merged.txt")

NameError: name 'load_data' is not defined

In [113]:
qids, questions, answers, labels = load_data(DATA_PATH + "dev.xml")

num_skipped 0


In [119]:
print(len((np.array(questions))))

len(np.unique(np.array(questions)))

1148


1038

In [63]:
prob, rel = generate_bm25_labels(qids, questions, answers)

In [123]:
map_score(qids, labels ,prob)

0.3474475787125922

In [124]:
mrr_score(qids, labels ,prob)

0.4067459356743531

In [11]:
'''Borrowed from https://github.com/aseveryn/deep-qa and 
https://github.com/shashankg7/Keras-CNN-QA
'''
import re
import os, string
import numpy as np
import pickle
import subprocess
from collections import defaultdict
import json
import pdb
import io

import sys
sys.path.insert(0, '../')

UNKNOWN_WORD_IDX = 0

def load_data(fname):
    lines = open(fname).readlines()
    qids, questions, answers, labels = [], [], [], []
    num_skipped = 0
    prev = ''
    qid2num_answers = {}
    for i, line in enumerate(lines):
        line = line.strip()

        qid_match = re.match('<QApairs id=\'(.*)\'>', line)

        if qid_match:
            qid = qid_match.group(1)
            qid2num_answers[qid] = 0

        if prev and prev.startswith('<question>'):
            question = line.lower().split('\t')

        label = re.match('^<(positive|negative)>', prev)
        if label:
            label = label.group(1)
            label = 1 if label == 'positive' else 0
            #line = line.translate(string.maketrans("",""), string.punctuation)
            line = re.sub("\d", ".", line)
            answer = line.lower().split('\t')
            if len(answer) > 60:
                num_skipped += 1
                continue
            labels.append(label)
            answers.append(answer)
            questions.append(question)
            qids.append(qid)
            qid2num_answers[qid] += 1
        prev = line
    # print sorted(qid2num_answers.items(), key=lambda x: float(x[0]))
    print('num_skipped', num_skipped)
    return qids, questions, answers, labels


# def compute_overlap_features(questions, answers, word2df=None, stoplist=None):
#   word2df = word2df if word2df else {}
#   stoplist = stoplist if stoplist else set()
#   feats_overlap = []
#   for question, answer in zip(questions, answers):
#     # q_set = set(question)
#     # a_set = set(answer)
#     q_set = set([q for q in question if q not in stoplist])
#     a_set = set([a for a in answer if a not in stoplist])
#     word_overlap = q_set.intersection(a_set)
#     # overlap = float(len(word_overlap)) / (len(q_set) * len(a_set) + 1e-8)
#     overlap = float(len(word_overlap)) / (len(q_set) + len(a_set))

#     # q_set = set([q for q in question if q not in stoplist])
#     # a_set = set([a for a in answer if a not in stoplist])
#     word_overlap = q_set.intersection(a_set)
#     df_overlap = 0.0
#     for w in word_overlap:
#       df_overlap += word2df[w]
#     df_overlap /= (len(q_set) + len(a_set))

#     feats_overlap.append(np.array([
#                          overlap,
#                          df_overlap,
#                          ]))
#   return np.array(feats_overlap)


# def compute_overlap_idx(questions, answers, stoplist, q_max_sent_length, a_max_sent_length):
#   stoplist = stoplist if stoplist else []
#   feats_overlap = []
#   q_indices, a_indices = [], []
#   for question, answer in zip(questions, answers):
#     q_set = set([q for q in question if q not in stoplist])
#     a_set = set([a for a in answer if a not in stoplist])
#     word_overlap = q_set.intersection(a_set)

#     q_idx = np.ones(q_max_sent_length) * 2
#     for i, q in enumerate(question):
#       value = 0
#       if q in word_overlap:
#         value = 1
#       q_idx[i] = value
#     q_indices.append(q_idx)

#     #### ERROR
#     # a_idx = np.ones(a_max_sent_length) * 2
#     # for i, q in enumerate(question):
#     #   value = 0
#     #   if q in word_overlap:

#     a_idx = np.ones(a_max_sent_length) * 2
#     for i, a in enumerate(answer):
#       value = 0
#       if a in word_overlap:
#         value = 1
#       a_idx[i] = value
#     a_indices.append(a_idx)

#   q_indices = np.vstack(q_indices).astype('int32')
#   a_indices = np.vstack(a_indices).astype('int32')

#   return q_indices, a_indices


# def compute_dfs(docs):
#   word2df = defaultdict(float)
#   for doc in docs:
#     for w in set(doc):
#       word2df[w] += 1.0
#   num_docs = len(docs)
#   for w, value in word2df.items():
#     word2df[w] /= np.math.log(num_docs / value)
#   return word2df


# def add_to_vocab(data, alphabet):
#   for sentence in data:
#     for token in sentence:
#       alphabet.add(token)


# def convert2indices(data, alphabet, dummy_word_idx, max_sent_length=40):
#   data_idx = []
#   for sentence in data:
#     ex = np.ones(max_sent_length) * dummy_word_idx
#     for i, token in enumerate(sentence):
#       idx = alphabet.get(token, UNKNOWN_WORD_IDX)
#       ex[i] = idx
#     data_idx.append(ex)
#   data_idx = np.array(data_idx).astype('int32')
#   return data_idx

def gen_vocab(data):
    vocab = defaultdict(int)
    vocab_idx = 1
    for component in data:
        for text in component:
            for token in text:
                if token not in vocab:
                    vocab[token] = vocab_idx
                    vocab_idx += 1

    vocab['UNK'] = len(vocab)
    f = open(OUTPUT_PATH + 'vocab.json', 'w')
    json.dump(vocab, f)
    return vocab


def get_maxlen(data):
    return max(map(lambda x:len(x), data))

def gen_seq(data, vocab, max_len):
    X = []

    for text in data:
        temp = [0] * max_len
        temp[:len(text)] = map(lambda x:vocab.get(x, vocab['UNK']), text)
        X.append(temp)

    return np.array(X)


# stoplist = set([line.strip() for line in open('en.txt')])
# import string
# punct = set(string.punctuation)
# stoplist.update(punct)
stoplist = None

train = DATA_PATH + 'train.xml'
train_all = DATA_PATH + 'train-all.xml'
train_files = [train, train_all]

for train in train_files:
    print(train)

    dev = DATA_PATH + 'dev.xml'
    test = DATA_PATH + 'test.xml'

    train_basename = os.path.basename(train)
    name, ext = os.path.splitext(train_basename)
    outdir = OUTPUT_PATH + '{}'.format(name.upper())
    print('outdir', outdir)

    if not os.path.exists(outdir):
        os.makedirs(outdir)

    
    # all_fname = train
    all_fname = OUTPUT_PATH + "tmp\\trec-merged.txt"
    files = ' '.join([train, dev, test])
    a = subprocess.call("type {} > {}".format(files, all_fname), shell=True)

    # qids, questions, answers, labels = load_data(all_fname, stoplist)
    qids, questions, answers, labels = load_data(all_fname)
    vocab = gen_vocab([questions, answers])
    max_len_ques = get_maxlen(questions)
    max_len_ans = get_maxlen(answers)

    # Convert dev and test sets
    for fname in [train, dev, test]:
        print('fname is', fname)
        # qids, questions, answers, labels = load_data(fname, stoplist)
        qids, questions, answers, labels = load_data(fname)  
        X_ques = gen_seq(questions, vocab, max_len_ques)
        X_ans = gen_seq(answers, vocab, max_len_ans)
        # pdb.set_trace()

        # overlap_feats = compute_overlap_features(questions, answers, None)
        # print overlap_feats[:10]

        qids = np.array(qids)
        labels = np.array(labels).astype('int32')

        _, counts = np.unique(labels, return_counts=True)
        print(counts / float(np.sum(counts)))
        print("questions", len(np.unique(qids)))
        # print "questions", len(qids)
        print("pairs", len(labels))

        # stoplist = None
        #q_overlap_indices, a_overlap_indices = compute_overlap_idx(questions, answers, stoplist, q_max_sent_length, a_max_sent_length)
        # print q_overlap_indices[:3]
        # print a_overlap_indices[:3]

        #questions_idx = convert2indices(questions, alphabet, dummy_word_idx, q_max_sent_length)
        #answers_idx = convert2indices(answers, alphabet, dummy_word_idx, a_max_sent_length)
        #print 'answers_idx', answers_idx.shape
        #pdb.set_trace()
        basename, _ = os.path.splitext(os.path.basename(fname))
        np.save(os.path.join(outdir, '{}.qids.npy'.format(basename)), qids)
        np.save(os.path.join(outdir, '{}.questions.npy'.format(basename)), X_ques)
        np.save(os.path.join(outdir, '{}.answers.npy'.format(basename)), X_ans)
        np.save(os.path.join(outdir, '{}.labels.npy'.format(basename)), labels)
        # np.save(os.path.join(outdir, '{}.overlap_feats.npy'.format(basename)), overlap_feats)

        # np.save(os.path.join(outdir, '{}.q_overlap_indices.npy'.format(basename)), q_overlap_indices)
        # np.save(os.path.join(outdir, '{}.a_overlap_indices.npy'.format(basename)), a_overlap_indices)

C:\Users\dmytro.rybalko\Documents\impementations\Keras_PairCNN\jacana-qa-naacl2013-data-results\train.xml
outdir C:\Users\dmytro.rybalko\Documents\impementations\Keras_PairCNN\jacana-qa-naacl2013-data-results\preprocessed_data\TRAIN
num_skipped 0
fname is C:\Users\dmytro.rybalko\Documents\impementations\Keras_PairCNN\jacana-qa-naacl2013-data-results\train.xml
num_skipped 0
[0.92623993 0.07376007]
questions 93
pairs 4718
fname is C:\Users\dmytro.rybalko\Documents\impementations\Keras_PairCNN\jacana-qa-naacl2013-data-results\dev.xml
num_skipped 0
[0.80662021 0.19337979]
questions 81
pairs 1148
fname is C:\Users\dmytro.rybalko\Documents\impementations\Keras_PairCNN\jacana-qa-naacl2013-data-results\test.xml
num_skipped 0
[0.8127884 0.1872116]
questions 95
pairs 1517
C:\Users\dmytro.rybalko\Documents\impementations\Keras_PairCNN\jacana-qa-naacl2013-data-results\train-all.xml
outdir C:\Users\dmytro.rybalko\Documents\impementations\Keras_PairCNN\jacana-qa-naacl2013-data-results\preprocessed_d

In [45]:
'''Model utils'''
import os, sys

import gensim
from gensim.models.keyedvectors import KeyedVectors

import keras
from keras import backend as K
from keras import optimizers, regularizers
from keras.layers import Dense, Dropout
from keras.layers import Input, Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers.merge import Concatenate, Dot
from keras.layers import Multiply 
from keras.models import Sequential, Model

import numpy as np
import time

sys.path.insert(0, '../')

def load_embeddings(embedding_file, vocab):
    '''Load pre-learnt word embeddings.
    Return: embedding: embedding matrix with dim |vocab| x dim
            dim: dimension of the embeddings
            rand_count: number of words not in trained embedding
    '''
    print('Loading word vectors...')
    start = time.time()
    outdir = OUTPUT_PATH

    try:
        print('Trying to load from npy dump.')
        embedding = np.load(os.path.join(outdir, 'embedding.npy'))
        return embedding, embedding.shape[1], 'NA'
    except:
        print('Load from dump failed, reading from binary.')

    word_vectors = KeyedVectors.load_word2vec_format(
        embedding_file, binary=True)
    print('Loaded in %f seconds' %(time.time() - start))
    # Need to use the word vectors to make embeddings matrix
    # Get dimension for any word embedding
    dim = word_vectors['apple'].shape[0]
    
    # Initialize an embedding of |vocab| x dim
    # word -> embedding
    embedding = np.zeros((len(vocab), dim))
    # Take random values
    rand_vec = np.random.uniform(-0.25, 0.25, dim)
    # Count of words not having representations in our embedding file
    rand_count = 0

    for key, value in vocab.items():
        # Map word idx to its embedding vector.
        try:
            embedding[value] = word_vectors[key]
        except:
            embedding[value] = rand_vec
            rand_count += 1

    print('Total time for loading embedding: %f seconds' %(time.time() - start))
    print('Number of words not in trained embedding: %d' %(rand_count))

    np.save(os.path.join(outdir, 'embedding.npy'), embedding)
    return embedding, dim, rand_count


def counterfactual_risk_loss(weights):
    
    def compute_loss(y_true, y_pred):
        y_pred_prob = y_true*y_pred + (1 - y_true)*(1 - y_pred)
        return K.mean(weights * y_pred_prob, axis=-1)
        
    #y_true represents the weighting of policy defined by (sigma - lagrange_mult)/control_propensity
    return compute_loss

def cnn_model(embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters=100):
    '''Neural architecture as mentioned in the original paper.'''
    print('Preparing model with the following parameters: ')
    print('''embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters: ''')
    print(embed_dim, max_ques_len, max_ans_len, vocab_size, embedding.shape, 
          addit_feat_len, no_conv_filters)

    # Prepare layers for Question
    input_q = Input(shape=(max_ques_len,), name='ques_input')
    # print input_q.name
    # we will load embedding values from corpus here.
    embed_q = Embedding(input_dim=vocab_size, output_dim=embed_dim,
                        input_length=max_ques_len,
                        weights=[embedding], trainable=False)(input_q)
    # Padding means, if input size is 32x32, output will also be 32x32, i.e,
    # the dimensions will not reduce
    conv_q = Conv1D(filters=no_conv_filters, kernel_size=5, strides=1, padding='same',
                    activation='relu',
                    kernel_regularizer=regularizers.l2(1e-5),
                    name='ques_conv')(embed_q)
    # also referenced as x(q) in paper
    pool_q = GlobalMaxPooling1D(name='ques_pool')(conv_q)

    # Prepare layers for Answer
    input_a = Input(shape=(max_ans_len,))
    # print input_a.name
    embed_a = Embedding(input_dim=vocab_size, output_dim=embed_dim,
                        input_length=max_ans_len,
                        weights=[embedding], trainable=False)(input_a)
    conv_a = Conv1D(filters=no_conv_filters, kernel_size=5, strides=1, padding='same',
                    activation='relu',
                    kernel_regularizer=regularizers.l2(1e-5))(embed_a)
    pool_a = GlobalMaxPooling1D()(conv_a)

    # M or the similarity layer here
    # Paper: x_d_dash = M.x_d
    x_a = Dense(no_conv_filters, use_bias=False,
                kernel_regularizer=regularizers.l2(1e-4))(pool_a)
    sim = Dot(axes=-1)([pool_q, x_a])

    # Input additional features.
    input_addn_feat = Input(shape=(addit_feat_len, ), name='input_addn_feat')
    
    # Combine Question, sim, Answer pooled outputs and additional input features
    join_layer = keras.layers.concatenate([pool_q, sim, pool_a,
                                            input_addn_feat])

    # hidden_units = join_layer.output_shape[1]
    hidden_units = no_conv_filters + 1 + no_conv_filters + addit_feat_len
    # Using relu here too? Not mentioned in the paper.
    hidden_layer = Dense(hidden_units,
                         kernel_regularizer=regularizers.l2(1e-4),
                         activation = 'relu')(join_layer)
    hidden_layer = Dropout(0.5)(hidden_layer)

    # Final Softmax Layer, add regularizer here too?
    softmax = Dense(1, activation='sigmoid')(hidden_layer)

    input_weights = Input(shape=(1, ), name = 'input_weights')
    model = Model(inputs=[input_q, input_a, input_addn_feat, input_weights], outputs=softmax)
    print(model.summary())

    adadelta = optimizers.Adadelta(rho=0.95, epsilon=1e-06, lr =5.0)
    #adam = optimizers.Adam(epsilon = 1e-01)
    # TODO: Use map_score as a metric here?
    model.compile(optimizer=adadelta,
                  loss=counterfactual_risk_loss(input_weights),
                  metrics=['accuracy'])

    return model
    # model.fit([data_a, data_b], labels, epochs=10)

def main():
    '''Main'''
    cnn_model(50, 33, 40, 5000, np.random.rand(5000, 50))

In [9]:
# Used from https://github.com/shashankg7/Keras-CNN-QA
import numpy as np
import random
import pdb
import math

def batch_gen(X, batch_size):
    n_batches = X.shape[0]/float(batch_size)
    n_batches = int(math.ceil(n_batches))
    end = int(X.shape[0]/float(batch_size)) * batch_size
    n = 0
    for i in range(0,n_batches):
        if i < n_batches - 1: 
            if len(X.shape) > 1:
                batch = X[i*batch_size:(i+1) * batch_size, :]
                yield batch
            else:
                batch = X[i*batch_size:(i+1) * batch_size]
                yield batch
        
        else:
            if len(X.shape) > 1:
                batch = X[end: , :]
                n += X[end:, :].shape[0]
                yield batch
            else:
                batch = X[end:]
                n += X[end:].shape[0]
                yield batch
    

#if __name__ == "__main__":
#    X = np.random.rand(31223, 300)
#    for batch in batch_gen(X, 21):
#        print(batch.shape)

In [10]:
# coding: utf-8
'''Utils'''
from collections import defaultdict
import json
import numpy as np
from sklearn import metrics

def ap_score(cands):
    '''cands: (predicted_scores, actual_labels)
    Using: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html
    It uses roc-auc and then computes avg-prec.
    '''
    y_true, y_pred = map(list, zip(*cands))
    # print y_true, y_pred
    count = 0
    score = 0
    for i, (y_true, y_pred) in enumerate(cands):
        if y_true > 0:
            count += 1.0
            score += count / (i + 1.0)
    return score / (count + 1e-6)
    # return metrics.average_precision_score(y_true, y_pred)

def map_score(qids, labels, preds):
    '''Method that computes Mean Average Precision for the given input.
    Authors use their custom method to train. Actual benchmark is done using TREC eval.
    Original Code:
    https://github.com/aseveryn/deep-qa/blob/master/run_nnet.py#L403
    Read more about it:
    https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a/sklearn/metrics/ranking.py#L107
    https://makarandtapaswi.wordpress.com/2012/07/02/intuition-behind-average-precision-and-map/
    http://fastml.com/what-you-wanted-to-know-about-mean-average-precision/
    '''

    qid_2_cand = defaultdict(list)
    for qid, label, pred in zip(qids, labels, preds):
        assert pred >= 0 and pred <= 1
        qid_2_cand[qid].append((label, pred))

    avg_precs = []
    for qid, cands in qid_2_cand.items():
        # get average prec score for all cands of qid
        avg_prec = ap_score(sorted(cands, reverse=True, key=lambda x: x[1]))
        avg_precs.append(avg_prec)
        
    return sum(avg_precs) / len(avg_precs)


def calculate_mrr_score(cands):
    
    y_true, y_pred = map(list, zip(*cands))
    for i, (y_true, y_pred) in enumerate(cands):
        if y_true > 0:
            return 1./(i + 1)
    return 0

def mrr_score(qids, labels, preds):

    qid_2_cand = defaultdict(list)
    for qid, label, pred in zip(qids, labels, preds):
        assert pred >= 0 and pred <= 1
        qid_2_cand[qid].append((label, pred))

    mrr_score = 0
    for qid, cands in qid_2_cand.items():
        # get average prec score for all cands of qid
        mrr_score += calculate_mrr_score(sorted(cands, reverse=True, key=lambda x: x[1]))    
    return mrr_score/len(qid_2_cand)

def load_json(file_path):
    '''Huh?'''
    return json.load(open(file_path, 'r'))

In [61]:
'''Main file to run the setup.'''
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import sys

from keras.callbacks import TensorBoard
import numpy as np
import pandas as pd
import sklearn
import subprocess
import tensorflow as tf
# import tqdm

sys.path.insert(0, '../')

def write_log(callback, names, logs, batch_no):
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()
        
def generate_weights(qids, questions, answers, labels, lagrange_mult = 0.9, variance = 0.001):
    probs, relevances = generate_bm25_labels(qids, questions, answers)
    loss = (1 + np.array(labels)) * np.random.normal(loc = labels - relevances,
                                                     scale = variance,
                                                     size = np.array(qids).shape[0])
    loss = np.power(loss, 2) - lagrange_mult
    return np.clip(loss/probs, -1000, 1000), relevances

#def generate_weights(qids, questions, answers, labels, lagrange_mult = 0.9, variance = 0.1):
#    probs, relevances = generate_bm25_labels(qids, questions, answers)
#    loss = relevances - lagrange_mult
#    return loss/probs

def train_model(mode):
    '''Train the model.
    1. Read numpy arrays for input data
    2. Batch train the model
    3. Calculate map scores using our method.
    4. Dump predicted values in csv format for evaluation using Trec-eval
    '''
    if mode not in ['TRAIN-ALL', 'TRAIN']:
        print('Invalid mode')
        return

    print('Training on %s set' %(mode))
    data_dir = os.path.join(OUTPUT_PATH, mode)

    # Load train set.
    q_train = np.load(os.path.join(data_dir, '%s.questions.npy' %(mode.lower())))
    a_train = np.load(os.path.join(data_dir, '%s.answers.npy' %(mode.lower())))
    y_train = np.load(os.path.join(data_dir, '%s.labels.npy' %(mode.lower())))
    qids_train = np.load(os.path.join(data_dir, '%s.qids.npy' %(mode.lower())))
    addn_feat_train = np.zeros(y_train.shape)
    
    weights, relevances = generate_weights(qids_train, q_train, a_train, y_train)
    
    print('''q_train.shape, a_train.shape, y_train.shape, qids_train.shape,
             addn_feat_train.shape: ''')
    print(q_train.shape, q_train.shape, y_train.shape, qids_train.shape,
          addn_feat_train.shape)

    # Load dev and test sets.
    q_dev = np.load(os.path.join(data_dir, 'dev.questions.npy'))
    a_dev = np.load(os.path.join(data_dir, 'dev.answers.npy'))
    y_dev = np.load(os.path.join(data_dir, 'dev.labels.npy'))
    qids_dev = np.load(os.path.join(data_dir, 'dev.qids.npy'))
    addn_feat_dev = np.zeros(y_dev.shape)

    q_test = np.load(os.path.join(data_dir, 'test.questions.npy'))
    a_test = np.load(os.path.join(data_dir, 'test.answers.npy'))
    y_test = np.load(os.path.join(data_dir, 'test.labels.npy'))
    qids_test = np.load(os.path.join(data_dir, 'test.qids.npy'))
    addn_feat_test = np.zeros(y_test.shape)

    vocab = load_json(VOCAB_PATH)
    
    max_ques_len = q_train.shape[1]
    max_ans_len = a_train.shape[1]
    embedding, embed_dim, _ = load_embeddings(EMBEDDING_PATH, vocab)
    # Only zeros
    addit_feat_len = 1
    if addn_feat_train.ndim > 1:
        addit_feat_len = addn_feat_train.shape[1]

    # Get model
    cnn_model_instance = cnn_model(embed_dim, max_ques_len, max_ans_len,
                                len(vocab), embedding,
                                addit_feat_len=addit_feat_len)

    def testing_things():
        ques_conv = cnn_model_instance.get_layer('ques_conv')
        print(ques_conv)
        print(ques_conv.input, ques_conv.output, ques_conv.input_shape,
              ques_conv.output_shape)
        ques_pool = cnn_model_instance.get_layer('ques_pool')
        print(ques_pool)
        print(ques_pool.input_shape, ques_pool.output_shape)
    # testing_things()
    
    bs = BATCH_SIZE
    np.set_printoptions(threshold=np.nan)
    # np.seterr(divide='ignore', invalid='ignore')
    # Train manually, epoch by epoch
    # TODO: Add tqdm
    log_path = './logs'
    callback = TensorBoard(log_path)
    callback.set_model(cnn_model_instance)
    train_names = ['train_loss', 'train_acc']
    dev_names = ['dev_loss', 'dev_acc']

    y_pred = cnn_model_instance.predict([q_train, a_train, addn_feat_train, np.ones(shape = len(q_train))])
    # print y_pred
    # print y_pred.shape, np.where(y_pred == 0)[0].shape
    train_acc = sklearn.metrics.roc_auc_score(y_train, y_pred)
    # TODO: add checking if best accuracy has been reached
    train_map = map_score(qids_dev, y_train, y_pred)
    train_mrr = mrr_score(qids_dev, y_train, y_pred)
    print('INITIAL TRAIN AUC: %f, MAP: %f, MRR: %f' %( train_acc, train_map, train_mrr))
    models = []
    for epoch in range(EPOCHS):
        print('EPOCH', epoch)
        # Obtain a shuffled batch of the samples
        q_train, a_train, y_train, addn_feat_train = sklearn.utils.shuffle(
            q_train, a_train, y_train, addn_feat_train,
            random_state=RANDOM_STATE)
        # print y_train.shape, np.where(y_train == 1)[0].shape
        # break
        # history = cnn_model.fit([q_train, a_train], y_train, batch_size=bs,
                                  # epochs=5, shuffle=False)
        # print 'Loss, Acc: %f %f' %(loss, acc)
        batch_no = 0
        
        y_pred = cnn_model_instance.predict([q_dev, a_dev, addn_feat_dev, np.ones(shape = len(q_dev))])
        # print y_pred
        # print y_pred.shape, np.where(y_pred == 0)[0].shape
        dev_acc = sklearn.metrics.roc_auc_score(y_dev, y_pred)
        # TODO: add checking if best accuracy has been reached
        dev_map = map_score(qids_dev, y_dev, y_pred)
        dev_mrr = mrr_score(qids_dev, y_dev, y_pred)
        print('Start, Dev AUC: %f, MAP: %f, MRR: %f' %(dev_acc, dev_map, dev_mrr))
                
        for b_q_train, b_a_train, b_y_train, b_addn_feat_train, b_weights, b_qids_train, b_relevances in zip(
                batch_gen(q_train, bs), batch_gen(a_train, bs),
                batch_gen(y_train, bs), batch_gen(addn_feat_train, bs),
                batch_gen(weights, bs), batch_gen(qids_train, bs),
                batch_gen(relevances, bs)):
        
            logs = cnn_model_instance.train_on_batch(
                [b_q_train, b_a_train, b_addn_feat_train, b_weights], b_relevances)
            if batch_no%10 == 0:
                write_log(callback, train_names, logs, batch_no*(epoch+1))
                y_pred = cnn_model_instance.predict([q_dev, a_dev, addn_feat_dev, np.ones(shape = len(q_dev))])
                # print y_pred
                # print y_pred.shape, np.where(y_pred == 0)[0].shape
                dev_acc = sklearn.metrics.roc_auc_score(y_dev, y_pred)
                # TODO: add checking if best accuracy has been reached
                dev_map = map_score(qids_dev, y_dev, y_pred)
                dev_mrr = mrr_score(qids_dev, y_dev, y_pred)
                print('Batch %f, Dev AUC: %f, MAP: %f, MRR: %f' %(batch_no, dev_acc, dev_map, dev_mrr))
                y_pred = cnn_model_instance.predict([q_test, a_test, addn_feat_test, np.ones(shape = len(q_test))])
                test_acc = sklearn.metrics.roc_auc_score(y_test, y_pred)
                test_map = map_score(qids_test, y_test, y_pred)
                test_mrr = mrr_score(qids_test, y_test, y_pred)
                print('Batch %f, Test AUC: %f, MAP: %f, MRR: %f' %(batch_no, test_acc, test_map, test_mrr))
                y_pred = cnn_model_instance.predict([q_train, a_train, addn_feat_train, np.ones(shape = len(q_train))])
                # print y_pred
                # print y_pred.shape, np.where(y_pred == 0)[0].shape
                train_acc = sklearn.metrics.roc_auc_score(y_train, y_pred)
                # TODO: add checking if best accuracy has been reached
                train_map = map_score(qids_dev, y_train, y_pred)
                train_mrr = mrr_score(qids_dev, y_train, y_pred)
                print('Batch %f, TRAIN AUC: %f, MAP: %f, MRR: %f' %(batch_no, train_acc, train_map, train_mrr))
            batch_no += 1



        #y_pred_train = cnn_model_instance.predict([q_train, a_train, addn_feat_train, np.ones(shape = len(q_train))])
        #train_acc = sklearn.metrics.roc_auc_score(y_dev, y_pred)
        #train_map = map_score(qids_dev, y_dev, y_pred_train)
        #train_mrr = mrr_score(qids_dev, y_dev, y_pred_train)
        #print('Train AUC: %f, MAP: %f, MRR: %f' %(dev_acc, dev_map, dev_mrr))    
            
        # Predict result on dev set
        y_pred = cnn_model_instance.predict([q_dev, a_dev, addn_feat_dev, np.ones(shape = len(q_dev))])
        # print y_pred
        # print y_pred.shape, np.where(y_pred == 0)[0].shape
        dev_acc = sklearn.metrics.roc_auc_score(y_dev, y_pred)
        # TODO: add checking if best accuracy has been reached
        dev_map = map_score(qids_dev, y_dev, y_pred)
        dev_mrr = mrr_score(qids_dev, y_dev, y_pred)
        # write_log(callback, dev_names, logs, epoch)
        print(' Dev AUC: %f, MAP: %f, MRR: %f' %(dev_acc, dev_map, dev_mrr))
        models += [cnn_model_instance]

    y_pred = cnn_model_instance.predict([q_test, a_test, addn_feat_test, np.ones(shape = len(q_test))])
    test_acc = sklearn.metrics.roc_auc_score(y_test, y_pred)
    test_map = map_score(qids_test, y_test, y_pred)
    test_mrr = mrr_score(qids_test, y_test, y_pred)
    print('Test AUC: %f, MAP: %f, MRR: %f' %(test_acc, test_map, test_mrr))

    # Dump data for trec eval
    N = len(y_pred)
    nnet_outdir = OUTPUT_PATH + 'output\\'

    df_submission = pd.DataFrame(index=np.arange(N), columns=['qid', 'iter', 'docno', 'rank', 'sim', 'run_id'])
    df_submission['qid'] = qids_test
    df_submission['iter'] = 0
    df_submission['docno'] = np.arange(N)
    df_submission['rank'] = 0
    df_submission['sim'] = y_pred
    df_submission['run_id'] = 'nnet'
    df_submission.to_csv(os.path.join(nnet_outdir, 'submission.txt'), header=False, index=False, sep=' ')

    df_gold = pd.DataFrame(index=np.arange(N), columns=['qid', 'iter', 'docno', 'rel'])
    df_gold['qid'] = qids_test
    df_gold['iter'] = 0
    df_gold['docno'] = np.arange(N)
    df_gold['rel'] = y_test
    df_gold.to_csv(os.path.join(nnet_outdir, 'gold.txt'), header=False, index=False, sep=' ')

    subprocess.call("/bin/sh eval/run_eval.sh '{}'".format(nnet_outdir), shell=True)
    return cnn_model_instance, models

In [62]:
mode_with_batch_train, models = train_model(mode='TRAIN')

Training on TRAIN set
q_train.shape, a_train.shape, y_train.shape, qids_train.shape,
             addn_feat_train.shape: 
(4718, 33) (4718, 33) (4718,) (4718,) (4718,)
Loading word vectors...
Trying to load from npy dump.
Preparing model with the following parameters: 
embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters: 
50 33 40 52051 (52051, 50) 1 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 33)           0                                            
_____________________________________________________________________________________________

Batch 90.000000, Dev AUC: 0.383131, MAP: 0.478802, MRR: 0.486458
Batch 90.000000, Test AUC: 0.329651, MAP: 0.444852, MRR: 0.444772
Batch 90.000000, TRAIN AUC: 0.461947, MAP: 0.177040, MRR: 0.182660
 Dev AUC: 0.383136, MAP: 0.478802, MRR: 0.486458
EPOCH 2
Start, Dev AUC: 0.383136, MAP: 0.478802, MRR: 0.486458
Batch 0.000000, Dev AUC: 0.383146, MAP: 0.478802, MRR: 0.486458
Batch 0.000000, Test AUC: 0.329578, MAP: 0.444809, MRR: 0.444772
Batch 0.000000, TRAIN AUC: 0.461884, MAP: 0.138594, MRR: 0.139821
Batch 10.000000, Dev AUC: 0.383058, MAP: 0.478764, MRR: 0.486458
Batch 10.000000, Test AUC: 0.329432, MAP: 0.444733, MRR: 0.444772
Batch 10.000000, TRAIN AUC: 0.461465, MAP: 0.138576, MRR: 0.139804
Batch 20.000000, Dev AUC: 0.382990, MAP: 0.478683, MRR: 0.486426
Batch 20.000000, Test AUC: 0.328973, MAP: 0.444598, MRR: 0.444772
Batch 20.000000, TRAIN AUC: 0.461574, MAP: 0.138575, MRR: 0.139804
Batch 30.000000, Dev AUC: 0.382902, MAP: 0.478603, MRR: 0.486396
Batch 30.000000, Test AUC: 0.32883

Batch 80.000000, Test AUC: 0.327950, MAP: 0.443612, MRR: 0.442761
Batch 80.000000, TRAIN AUC: 0.454278, MAP: 0.111983, MRR: 0.123048
Batch 90.000000, Dev AUC: 0.383729, MAP: 0.488041, MRR: 0.498537
Batch 90.000000, Test AUC: 0.328010, MAP: 0.443595, MRR: 0.442761
Batch 90.000000, TRAIN AUC: 0.454452, MAP: 0.111988, MRR: 0.123048
 Dev AUC: 0.383622, MAP: 0.488041, MRR: 0.498537
EPOCH 6
Start, Dev AUC: 0.383622, MAP: 0.488041, MRR: 0.498537
Batch 0.000000, Dev AUC: 0.383608, MAP: 0.488041, MRR: 0.498537
Batch 0.000000, Test AUC: 0.328039, MAP: 0.443595, MRR: 0.442761
Batch 0.000000, TRAIN AUC: 0.454548, MAP: 0.124220, MRR: 0.133258
Batch 10.000000, Dev AUC: 0.383554, MAP: 0.488039, MRR: 0.498537
Batch 10.000000, Test AUC: 0.328036, MAP: 0.443595, MRR: 0.442761
Batch 10.000000, TRAIN AUC: 0.454751, MAP: 0.124222, MRR: 0.133258
Batch 20.000000, Dev AUC: 0.383491, MAP: 0.488078, MRR: 0.498537
Batch 20.000000, Test AUC: 0.328093, MAP: 0.443595, MRR: 0.442761
Batch 20.000000, TRAIN AUC: 0.454

In [60]:
models

In [44]:
mode_with_batch = train_model(mode='TRAIN-ALL')

Training on TRAIN-ALL set
q_train.shape, a_train.shape, y_train.shape, qids_train.shape,
             addn_feat_train.shape: 
(53417, 33) (53417, 33) (53417,) (53417,) (53417,)
Loading word vectors...
Trying to load from npy dump.
Preparing model with the following parameters: 
embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters: 
50 33 60 52051 (52051, 50) 1 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 33)           0                                            
____________________________________________________________________________________

Batch 900.000000, Dev AUC: 0.413227, MAP: 0.490907, MRR: 0.489402
Batch 1000.000000, Dev AUC: 0.412984, MAP: 0.491269, MRR: 0.491460
 Dev AUC: 0.413135, MAP: 0.491216, MRR: 0.491166
Start, Dev AUC: 0.413135, MAP: 0.491216, MRR: 0.491166
Batch 0.000000, Dev AUC: 0.413135, MAP: 0.491216, MRR: 0.491166
Batch 100.000000, Dev AUC: 0.413291, MAP: 0.491505, MRR: 0.491460
Batch 200.000000, Dev AUC: 0.413826, MAP: 0.492256, MRR: 0.491166
Batch 300.000000, Dev AUC: 0.414483, MAP: 0.491972, MRR: 0.491597
Batch 400.000000, Dev AUC: 0.414628, MAP: 0.491874, MRR: 0.491303
Batch 500.000000, Dev AUC: 0.414755, MAP: 0.491874, MRR: 0.491303
Batch 600.000000, Dev AUC: 0.415460, MAP: 0.491922, MRR: 0.491303
Batch 700.000000, Dev AUC: 0.416085, MAP: 0.491506, MRR: 0.490941
Batch 800.000000, Dev AUC: 0.415251, MAP: 0.490829, MRR: 0.488904
Batch 900.000000, Dev AUC: 0.416098, MAP: 0.490820, MRR: 0.488936
Batch 1000.000000, Dev AUC: 0.415373, MAP: 0.490823, MRR: 0.488936
 Dev AUC: 0.416015, MAP: 0.491631, MRR

Batch 800.000000, Dev AUC: 0.499460, MAP: 0.957818, MRR: 0.956790
Batch 900.000000, Dev AUC: 0.499460, MAP: 0.957818, MRR: 0.956790
Batch 1000.000000, Dev AUC: 0.505025, MAP: 0.847454, MRR: 0.833642
 Dev AUC: 0.494600, MAP: 0.918001, MRR: 0.905350
Test AUC: 0.494323, MAP: 0.908475, MRR: 0.890351


In [125]:
model_train = train_model(mode='TRAIN')

Training on TRAIN set
q_train.shape, a_train.shape, y_train.shape, qids_train.shape,
             addn_feat_train.shape: 
(4718, 33) (4718, 33) (4718,) (4718,) (4718,)
Loading word vectors...
Trying to load from npy dump.
Preparing model with the following parameters: 
embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters: 
50 33 40 52051 (52051, 50) 1 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 33)           0                                            
_____________________________________________________________________________________________

In [96]:
model_train_all = train_model(mode='TRAIN-ALL')

Training on TRAIN-ALL set
q_train.shape, a_train.shape, y_train.shape, qids_train.shape,
             addn_feat_train.shape: 
(53417, 33) (53417, 33) (53417,) (53417,) (53417,)
Loading word vectors...
Trying to load from npy dump.
Preparing model with the following parameters: 
embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters: 
50 33 60 52051 (52051, 50) 1 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 33)           0                                            
____________________________________________________________________________________

In [127]:
model_train_100 = train_model(mode='TRAIN')

Training on TRAIN set
q_train.shape, a_train.shape, y_train.shape, qids_train.shape,
             addn_feat_train.shape: 
(4718, 33) (4718, 33) (4718,) (4718,) (4718,)
Loading word vectors...
Trying to load from npy dump.
Preparing model with the following parameters: 
embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters: 
50 33 40 52051 (52051, 50) 1 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 33)           0                                            
_____________________________________________________________________________________________

 Dev AUC: 0.351969, MAP: 0.473208, MRR: 0.477436
 Dev AUC: 0.351804, MAP: 0.473275, MRR: 0.477436
 Dev AUC: 0.351741, MAP: 0.476791, MRR: 0.485662
 Dev AUC: 0.351723, MAP: 0.476695, MRR: 0.485368
 Dev AUC: 0.351658, MAP: 0.476596, MRR: 0.485368
 Dev AUC: 0.351609, MAP: 0.476598, MRR: 0.485386
 Dev AUC: 0.351551, MAP: 0.476598, MRR: 0.485386
 Dev AUC: 0.351643, MAP: 0.476600, MRR: 0.485386
 Dev AUC: 0.351697, MAP: 0.476599, MRR: 0.485386
 Dev AUC: 0.351809, MAP: 0.476626, MRR: 0.485386
 Dev AUC: 0.351765, MAP: 0.476626, MRR: 0.485386
 Dev AUC: 0.351823, MAP: 0.476626, MRR: 0.485386
 Dev AUC: 0.351998, MAP: 0.476626, MRR: 0.485386
 Dev AUC: 0.352066, MAP: 0.476336, MRR: 0.484791
 Dev AUC: 0.352096, MAP: 0.476339, MRR: 0.484810
 Dev AUC: 0.352353, MAP: 0.476313, MRR: 0.484783
 Dev AUC: 0.352495, MAP: 0.476328, MRR: 0.484805
 Dev AUC: 0.352446, MAP: 0.476316, MRR: 0.484805
 Dev AUC: 0.352572, MAP: 0.476242, MRR: 0.484805
Test AUC: 0.369236, MAP: 0.449116, MRR: 0.461226


In [ ]:
model_train_500 = train_model(mode='TRAIN')

Training on TRAIN set
q_train.shape, a_train.shape, y_train.shape, qids_train.shape,
             addn_feat_train.shape: 
(4718, 33) (4718, 33) (4718,) (4718,) (4718,)
Loading word vectors...
Trying to load from npy dump.
Preparing model with the following parameters: 
embed_dim, max_ques_len, max_ans_len, vocab_size, embedding,
              addit_feat_len, no_conv_filters: 
50 33 40 52051 (52051, 50) 1 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 33)           0                                            
_____________________________________________________________________________________________

 Dev AUC: 0.370775, MAP: 0.471705, MRR: 0.478097
 Dev AUC: 0.371242, MAP: 0.471773, MRR: 0.478097
 Dev AUC: 0.371427, MAP: 0.471763, MRR: 0.478097
 Dev AUC: 0.371529, MAP: 0.471793, MRR: 0.478097
 Dev AUC: 0.371743, MAP: 0.471790, MRR: 0.478097
 Dev AUC: 0.371782, MAP: 0.471793, MRR: 0.478097
 Dev AUC: 0.371870, MAP: 0.471807, MRR: 0.478097
 Dev AUC: 0.372069, MAP: 0.471890, MRR: 0.478129
 Dev AUC: 0.372030, MAP: 0.471847, MRR: 0.478135
 Dev AUC: 0.372132, MAP: 0.471900, MRR: 0.478135
 Dev AUC: 0.372196, MAP: 0.471823, MRR: 0.477963
 Dev AUC: 0.372312, MAP: 0.471499, MRR: 0.478009
 Dev AUC: 0.372434, MAP: 0.471462, MRR: 0.478018
 Dev AUC: 0.372512, MAP: 0.471458, MRR: 0.478018
 Dev AUC: 0.372882, MAP: 0.471587, MRR: 0.478498
 Dev AUC: 0.372969, MAP: 0.471458, MRR: 0.478198
 Dev AUC: 0.373120, MAP: 0.470432, MRR: 0.478257
 Dev AUC: 0.373134, MAP: 0.470385, MRR: 0.477846
 Dev AUC: 0.373183, MAP: 0.469301, MRR: 0.476817
 Dev AUC: 0.373266, MAP: 0.469368, MRR: 0.476817
 Dev AUC: 0.373271, 

 Dev AUC: 0.384984, MAP: 0.472586, MRR: 0.479583
 Dev AUC: 0.384970, MAP: 0.472600, MRR: 0.479583
 Dev AUC: 0.384960, MAP: 0.472601, MRR: 0.479720
 Dev AUC: 0.385057, MAP: 0.472527, MRR: 0.479583
 Dev AUC: 0.385091, MAP: 0.472628, MRR: 0.479720
 Dev AUC: 0.385213, MAP: 0.472368, MRR: 0.479171
 Dev AUC: 0.385296, MAP: 0.472368, MRR: 0.479171
 Dev AUC: 0.385349, MAP: 0.472360, MRR: 0.479171
 Dev AUC: 0.385456, MAP: 0.472384, MRR: 0.479171
 Dev AUC: 0.385695, MAP: 0.472407, MRR: 0.479171
 Dev AUC: 0.385753, MAP: 0.472311, MRR: 0.479034
 Dev AUC: 0.385870, MAP: 0.472407, MRR: 0.479171
 Dev AUC: 0.385913, MAP: 0.472434, MRR: 0.479171
 Dev AUC: 0.386069, MAP: 0.472425, MRR: 0.479171
 Dev AUC: 0.386089, MAP: 0.470372, MRR: 0.479198
 Dev AUC: 0.386293, MAP: 0.469705, MRR: 0.478885
 Dev AUC: 0.386390, MAP: 0.470238, MRR: 0.479503
 Dev AUC: 0.386468, MAP: 0.469790, MRR: 0.478874
 Dev AUC: 0.386269, MAP: 0.470028, MRR: 0.478885
 Dev AUC: 0.386366, MAP: 0.470025, MRR: 0.478885
 Dev AUC: 0.386439, 

In [92]:
np.array(questions)

array([list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['what', 'do', 'practitioners', 'of', 'wicca', 'worship', '?']),
       list(['how', 'many', 'followers', 'does', 'wicca', 'have', '?']),
       list(['how', 'many', 'followers', 'does', 'wicca', 'have', '?']),
       list(['what', 'is', 'florence', 'nightingale', 'famous', 'for', '?']),
      

In [48]:
data_dir = os.path.join(OUTPUT_PATH, 'TRAIN')
q_test = np.load(os.path.join(data_dir, 'train.questions.npy'))
a_test = np.load(os.path.join(data_dir, 'train.answers.npy'))
y_test = np.load(os.path.join(data_dir, 'train.labels.npy'))
qids_test = np.load(os.path.join(data_dir, 'train.qids.npy'))
addn_feat_test = np.zeros(y_test.shape)
y_pred = mode_with_batch.predict([q_test, a_test, addn_feat_test, np.ones(shape = len(q_test))])
test_acc = sklearn.metrics.roc_auc_score(y_test, y_pred)
test_map = map_score(qids_test, y_test, y_pred)
test_mrr = mrr_score(qids_test, y_test, y_pred)
print('Test AUC: %f, MAP: %f, MRR: %f' %(test_acc, test_map, test_mrr))

ValueError: Error when checking input: expected input_17 to have shape (60,) but got array with shape (40,)